In [1]:
import requests

In [2]:
import geopandas as gpd
import pandas as pd

## Open File

In [3]:
file_path = "data/maakond_shp/maakond.shp"
estonia_counties_gdf = gpd.read_file(file_path)

## Get Open Population data

In [4]:
r = requests.get("https://xgis.maaamet.ee/xgis2/service/14qni09/RV0222U.json")
data = r.json()

In [5]:
estonian_2018_df = pd.DataFrame(
    data["data"]["MK"]["0.2.0"].items(), columns=["MKOOD", "2018"]
)
estonian_2025_df = pd.DataFrame(
    data["data"]["MK"]["0.2.7"].items(), columns=["MKOOD", "2025"]
)

In [6]:
estonian_pop_df = estonian_2018_df.merge(estonian_2025_df, on="MKOOD", how="inner")

In [7]:
estonian_pop_df["DIFF"] = estonian_pop_df["2025"] - estonian_pop_df["2018"]
estonian_pop_df["PERCENT"] = (
    (estonian_pop_df["DIFF"] / estonian_pop_df["2018"]) * 100
).round(decimals=1)

## Merge data and export

In [8]:
estonian_pop_gdf = estonia_counties_gdf.merge(estonian_pop_df, on="MKOOD", how="left")

In [9]:
estonian_pop_gdf = estonian_pop_gdf.to_crs(3300)
estonian_pop_gdf.to_file("data/estonian_population.gpkg")

In [10]:
estonian_pop_gdf["2018"].sum()

np.int64(328618)

In [11]:
estonian_pop_gdf["2025"].sum()

np.int64(285819)